# Use pretrained backbone to get video embeddings

In [ ]:
import torch
import torchvision
import pytorchvideo

In [ ]:
torch.cuda.empty_cache()
#device = "cpu"
device = "cuda:1" if torch.cuda.is_available() else "cpu"
print("Device: " + device)
print(f"Devices count: {torch.cuda.device_count()}")

In [ ]:
from pathlib import Path

In [ ]:
import json
import pandas
import numpy

In [ ]:
from tqdm import tqdm

In [ ]:
from misc.utils_mvit import *

In [ ]:
data_path = Path("./data")
videos_path = data_path / "train_dataset"
metadata_path = data_path / "train.csv"

In [ ]:
filenames = walk_directory(videos_path)
filenames = sorted(filenames)

## Model

In [ ]:
video_transform = VideoTransform()

In [ ]:
#model_name = "mvit_v1_b"
model_name = "mvit_v2_s"
model = getattr(torchvision.models.video, model_name)(weights=torchvision.models.video.MViT_V2_S_Weights.KINETICS400_V1)

In [ ]:
model = model.to(device)
model = model.eval()

In [ ]:
embeddings, durations = encode_videos(
    videos_path,
    tqdm(filenames),
    model,
    video_transform,
    torchvision.models.video.MViT_V1_B_Weights.KINETICS400_V1.transforms(),
    device,
    batch_size=32
)

## Save embeddings

In [ ]:
import os

In [ ]:
embeddings_path = data_path / model_name
os.makedirs(embeddings_path, exist_ok=True)

In [ ]:
embeddings_path_torch = embeddings_path / "embeddings.pt"
embeddings_path_numpy = embeddings_path / "embeddings"
embeddings_uuid_path  = embeddings_path / "embeddings_uuid.csv"

In [ ]:
torch.save(embeddings, embeddings_path_torch)
numpy.savez(embeddings_path_numpy, embeddings.numpy())

In [ ]:
embeddings_uuid = pandas.DataFrame([filename.split('.')[0] for filename in filenames], columns=["uuid"])

In [ ]:
embeddings_uuid.to_csv(embeddings_uuid_path, index=False)